## Fetching Real-Time Crypto Data with Binance API

This notebook demonstrates how to connect to the Binance WebSocket API to stream real-time pricing data for multiple cryptocurrencies. We will use `websockets` to establish the connection and `pandas` to structure the incoming data.

---

### 1. Imports and Setup

First, we import the necessary libraries:
* `json`: To parse the incoming data from the API, which is in JSON format.
* `websocket`: To connect to the Binance WebSocket stream.
* `pandas`: To process and display the data in a structured DataFrame.

```python
import json
import websocket
import pandas as pd
```

---

### 2. Configuring the API Stream

We define the assets we want to track (**BTC**, **ETH**, and **SOL**) and format them according to the Binance API's requirements for 1-minute candlestick (`kline_1m`) data streams.

```python
assets = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]
assets = [coin.lower() + "@kline_1m" for coin in assets]
```

Next, we join these individual stream names into a single string, separated by `/`, to subscribe to all of them in one connection.

```python
assets = '/'.join(assets)
socket = "wss://[stream.binance.com:9443/stream?streams=](https://stream.binance.com:9443/stream?streams=)"+assets
```
The final `socket` URL looks like this: `wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1m/ethusdt@kline_1m/solusdt@kline_1m`

---

### 3. Processing Incoming Data

We define two functions to handle the data as it arrives from the WebSocket.

1.  `on_message(ws, message)`: This is a **callback function** that automatically runs every time a new message is received from the server. It loads the JSON message and passes it to our `manipulation` function.

2.  `manipulation(source)`: This function does the heavy lifting. It parses the raw data to extract the most relevant information:
    * The **closing price** (`['data']['k']['c']`)
    * The **event timestamp** (`['data']['E']`)
    * The **asset symbol** (`['data']['s']`)

    It then organizes this information into a clean pandas DataFrame and prints it.

```python
def on_message(we, message):
    message = json.loads(message)
    manipulation(message)

def manipulation(source):
    rel_data = source['data']['k']['c']
    evt_time = pd.to_datetime(source['data']['E'], unit='ms')
    df = pd.DataFrame(rel_data, columns=[source['data']['s']], index=[evt_time])

    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df
```

---

### 4. Running the Connection

Finally, we create a `WebSocketApp` instance, passing our connection `socket` and the `on_message` function. The `run_forever()` method starts the client, which will continuously listen for messages and print the processed dataframes in real-time until the script is stopped.

```python
ws = websocket.WebSocketApp(socket, on_message = on_message)
ws.run_forever()
```

In [86]:
# Importing Libraries

import json
import websocket
import pandas as pd


In [87]:
def on_message(we, message):
    message = json.loads(message)
    manipulation(message)


def manipulation(source):
    rel_data = source['data']['k']['c']
    evt_time = pd.to_datetime(source['data']['E'], unit='ms')
    df = pd.DataFrame(rel_data, columns=[source['data']['s']], index=[evt_time])

    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

assets = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]

assets = [coin.lower() + "@kline_1m" for coin in assets]
assets = '/'.join(assets)

socket = "wss://stream.binance.com:9443/stream?streams="+assets

In [88]:
# Displays the source

source

{'stream': 'solusdt@kline_1m',
 'data': {'e': 'kline',
  'E': 1760311760799,
  's': 'SOLUSDT',
  'k': {'t': 1760311740000,
   'T': 1760311799999,
   's': 'SOLUSDT',
   'i': '1m',
   'f': 1624851904,
   'L': 1624852314,
   'o': '196.25000000',
   'c': '196.21000000',
   'h': '196.26000000',
   'l': '196.12000000',
   'v': '224.38400000',
   'n': 411,
   'x': False,
   'q': '44023.53288000',
   'V': '86.08400000',
   'Q': '16888.60390000',
   'B': '0'}}}

In [89]:
# Executes the web socket

ws = websocket.WebSocketApp(socket, on_message = on_message)
ws.run_forever()

                timestamp  SOLUSDT
0 2025-10-13 00:34:04.803   196.99
                timestamp    BTCUSDT
0 2025-10-13 00:34:06.029  115118.35
                timestamp  ETHUSDT
0 2025-10-13 00:34:06.029  4143.16
                timestamp  SOLUSDT
0 2025-10-13 00:34:06.803    197.0
                timestamp    BTCUSDT
0 2025-10-13 00:34:08.018  115116.95
                timestamp  ETHUSDT
0 2025-10-13 00:34:08.018  4142.74
                timestamp  SOLUSDT
0 2025-10-13 00:34:08.796   197.01


True